In [ ]:
from jetcam.csi_camera import CSICamera

import numpy as np
import cv2
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widget = ipywidgets.Image(format='jpeg')
camera = CSICamera(width=300, height=300)
print("camera_connected")

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

In [3]:
# backSub = cv2.createBackgroundSubtractorMOG2()
backSub = cv2.createBackgroundSubtractorKNN()
def complex_background_subtraction(image):
    image = backSub.apply(image)
    return image

In [2]:
kernel = np.ones((3, 3), np.uint8)
def basic_background_subtraction(image):
    image = cv2.blur(image, (10, 10))
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower = np.array([0, 0, 50])
    upper = np.array([255, 75, 180])
    mask = cv2.inRange(hsv, lower, upper)
    mask = cv2.erode(mask, kernel, iterations=10)
    mask = cv2.dilate(mask, kernel, iterations=10)
    mask = cv2.bitwise_not(mask)
    return mask

In [3]:
def background_filter(image):
    # mask = complex_background_subtraction(image)
    mask = basic_background_subtraction(image)
    return mask

In [4]:
def get_contours(image, threshold=0):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    return contours

In [5]:
kernel = np.ones((3, 3), np.uint8)


def pre_processing(image):
    image = cv2.blur(image, (10, 10))
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower = np.array([0, 0, 50])
    upper = np.array([255, 75, 180])
    mask = cv2.inRange(hsv, lower, upper)
    mask = cv2.erode(mask, kernel, iterations=10)
    mask = cv2.dilate(mask, kernel, iterations=10)
    # mask = cv2.erode(mask,kernel,iterations = 10)
    mask = cv2.bitwise_not(mask)
    image = cv2.bitwise_and(image, image, mask=mask)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    # thresh = cv2.erode(thresh,kernel,iterations = 30)
    # thresh = cv2.dilate(thresh, kernel, iterations = 30)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    for contour in contours:
        if cv2.contourArea(contour) > 5000:
            cv2.drawContours(image, [contour], -1, (0, 255, 0), 3)

    return image

In [6]:
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 0.5
color = (255, 0, 0)
thickness = 2


def create_object_marker(image, area, x, y, w, h):
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
    cv2.putText(image, f"bottom: {h + y}", (x + 5, y + 15), font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.putText(image, f"center: {x + int(w / 2)}", (x + 5, y + 30), font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.putText(image, f"area: {area}", (x + 5, y + 45), font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.circle(image, (x + int(w / 2), h + y), 5, (0, 0, 255), -1)
    return image


In [7]:
tl = (100, 50)
bl = (0, 300)
tr = (200, 50)
br = (300, 300)

pts1 = np.float32([tl, bl, tr, br])
pts2 = np.float32([[0, 0], [0, 300], [300, 0], [300, 300]])
matrix = cv2.getPerspectiveTransform(pts1, pts2)


def update_image(change):
    image = change['new']
    image = cv2.warpPerspective(image, matrix, (300, 300))
    # image = pre_processing(image)
    mask = background_filter(image)
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    contours = get_contours(masked_image)

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 6000:
            cv2.drawContours(image, [contour], -1, (0, 255, 0), 3)
            x, y, w, h = cv2.boundingRect(contour)
            image = create_object_marker(image, area, x, y, w, h)

    # masked_img = cv2.bitwise_and(image, image, mask=mask)
    # contours = detect_objects(image)
    # if len(contours) > 0:
    #     cv2.drawContours(image, contours, -1, (0, 255, 0), 3)
    # gray = cv2.cvtColor(masked_img, cv2.COLOR_BGR2GRAY)
    # _,gray = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    image_widget.value = bgr8_to_jpeg(image)

In [9]:
camera.unobserve(update_image, names='value')

NameError: name 'update_image' is not defined

In [8]:
display(image_widget)
camera.running = True
camera.observe(update_image, names='value')

Image(value=b'', format='jpeg')

In [ ]:
car.steering = 0.0

In [ ]:
car.throttle = -0.35

In [ ]:
car.throttle = 0
